# Домашнее задание 4. Регрессия
**Дедлайн 23:59 07.12.2018**



# Задача
[Данные](https://www.kaggle.com/sohier/calcofi#bottle.csv) представляют собой различные показания океанической воды, взятые в разных местах. Задача состоит в исследовании того, насколько возможным является восстановление температуры воды по её химическим показателям

**Целевая переменная** - *T_degC* (температура воды в Цельсиях)

# Условие
В рамках данной задачи Вам предлагается попробовать следующее:
1. Предложите метрику качества Вашей модели. Нужны ли нам здесь ассиметричные метрики (аля quantile regression)? Объясните свой выбор.
2. Попробуйте различные модели регрессии для данной задачи. Выберите и затюньте наилучшую.
3. Какие из этих моделей требуют нормализации данных? 
4. Какое качество получилось? Можно ли говорить о какой-либо значимой зависимости температуры воды от её химических показателей?

# Решение

## 0. Загрузка и предобработка данных

In [6]:
import pandas as pd
df = pd.read_csv('calcofi/bottle.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (47,73) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df.isna().sum()/df.shape[0] # доля пропущенных значений

Cst_Cnt                0.000000
Btl_Cnt                0.000000
Sta_ID                 0.000000
Depth_ID               0.000000
Depthm                 0.000000
T_degC                 0.012676
Salnty                 0.054753
O2ml_L                 0.195016
STheta                 0.060922
O2Sat                  0.235400
Oxy_µmol/Kg            0.235407
BtlNum                 0.862791
RecInd                 0.000000
T_prec                 0.012676
T_qual                 0.973259
S_prec                 0.054753
S_qual                 0.913381
P_qual                 0.220969
O_qual                 0.786468
SThtaq                 0.923892
O2Satq                 0.748172
ChlorA                 0.739529
Chlqua                 0.260963
Phaeop                 0.739530
Phaqua                 0.260958
PO4uM                  0.522101
PO4q                   0.477621
SiO3uM                 0.590581
SiO3qu                 0.409310
NO2uM                  0.609677
                         ...   
MeanAs  

Данные очень сырые. Выкинем те объекты, которые содержат пропущенную целевую переменную

In [8]:
df = df[~df['T_degC'].isnull()]

In [9]:
import numpy as np
df_num = df.select_dtypes(include=[np.float]) # оставим только числовые признаки
df_num.head()

,T_degC,Salnty,O2ml_L,STheta,O2Sat,Oxy_µmol/Kg,BtlNum,T_prec,T_qual,S_prec,...,R_NH4,R_CHLA,R_PHAEO,R_SAMP,DIC1,DIC2,TA1,TA2,pH2,pH1
0,10.50,33.440,NaN,25.649,NaN,NaN,NaN,1.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10.46,33.440,NaN,25.656,NaN,NaN,NaN,2.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10.46,33.437,NaN,25.654,NaN,NaN,NaN,2.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10.45,33.420,NaN,25.643,NaN,NaN,NaN,2.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10.45,33.421,NaN,25.643,NaN,NaN,NaN,2.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.cross_validation import train_test_split

X, y = df_num.drop('T_degC',  1), df_num['T_degC']
cols = X.columns
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(X)
X = pd.DataFrame(imp.transform(X), columns=cols)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.85)

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Предложите метрику качества Вашей модели. Нужны ли нам здесь ассиметричные метрики (аля quantile regression)? Объясните свой выбор.

Впервые слышу про ассиметричные метрики. Что удалось найти про quantile regression – это тип регрессионного анализа, используемого в статистике и эконометрике. В то время как метод наименьших квадратов приводит к оценкам условного среднего переменной ответа при определенных значениях предикторных переменных, регрессия квантилей направлена на оценку либо условных медианных, либо других квантилей переменной ответа. По существу, квантильная регрессия является продолжением линейной регрессии, и мы ее используем, когда условия линейной регрессии неприменимы.

Мне бы с классикой разобраться: RMSE, MSE, MAE, MAPE. Их и буду использовать.

# Попробуйте различные модели регрессии для данной задачи. Выберите и затюньте наилучшую.

In [11]:
from tqdm import tqdm
from sklearn import linear_model
from sklearn import tree
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error

models = [
    linear_model.LinearRegression(),
    linear_model.Ridge(alpha = .5),
    linear_model.Lasso(alpha = 0.1),
    tree.DecisionTreeRegressor()
]

for m in tqdm(models):
    m.fit(X_train, y_train)
    print(m.__class__)
    print(mean_absolute_error(m.predict(X_test), y_test))
    print(mean_squared_error(m.predict(X_test), y_test))
    print(mean_squared_log_error(m.predict(X_test), y_test))
    print(median_absolute_error(m.predict(X_test), y_test))
    print('\n')


  0%|          | 0/4 [00:00<?, ?it/s]/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:509: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)
 25%|██▌       | 1/4 [00:03<00:11,  3.95s/it]

<class 'sklearn.linear_model.base.LinearRegression'>
0.00012552194036876739
3.4029877596646766e-07
2.9974716049549873e-09
2.0307142822417745e-05




 50%|█████     | 2/4 [00:05<00:05,  2.50s/it]

<class 'sklearn.linear_model.ridge.Ridge'>
0.0001257192915812253
3.402130513758173e-07
2.9971951085585946e-09
2.0424693635057167e-05




 75%|███████▌  | 3/4 [00:18<00:06,  6.22s/it]

<class 'sklearn.linear_model.coordinate_descent.Lasso'>
0.04262738222041654
0.004747983572382502
8.148189168191975e-05
0.02567330464485451


<class 'sklearn.tree.tree.DecisionTreeRegressor'>
0.00014297040380778918
6.589621531507673e-07
5.669897604286865e-09


100%|██████████| 4/4 [00:37<00:00,  9.38s/it]

5.649717586209135e-06




Победил: class 'sklearn.linear_model.coordinate_descent.Lasso

In [31]:
from sklearn.grid_search import GridSearchCV

model = linear_model.Lasso()

params_grid = {'alpha': [0.1, 0.4, 1.0],
              'max_iter': [i for i in range(100,1300,300)]}
gs = GridSearchCV(model, params_grid, scoring='neg_mean_absolute_error', verbose=0, n_jobs=-1)
gs.fit(X_train, y_train)

model = gs.best_estimator_

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of

In [32]:
model

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=400,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

# Какие из этих моделей требуют нормализации данных?

Во всех линейных моделях (LinearRegression, Ridge, Lasso) признаки нужно стандартизовать (вычесть среднее и поделить на стандартное отклонение, чтобы среднее стало нулевым, а стандартное отклонение единицей). Потому что:
Одно из наиболее важных допущений при работе с линейными моделями, параметры которых оцениваются методом наименьших квадратов, состоит в том, что остатки модели независимы (т.е. не коррелируют) и имеют нормальное распределение со средним значением 0 и некоторым фиксированным стандартным отклонением. Попробуем оценить качество до и после стандартизации.


In [33]:
model.fit(X_train, y_train)
print(mean_absolute_error(m.predict(X_test), y_test))
print(mean_squared_error(m.predict(X_test), y_test))
print(mean_squared_log_error(m.predict(X_test), y_test))
print(median_absolute_error(m.predict(X_test), y_test))
print('\n')

0.00014297040380778918
6.589621531507673e-07
5.669897604286865e-09
5.649717586209135e-06




In [34]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
scaler.fit(X_test)
X_test = scaler.transform(X_test)

In [35]:
model.fit(X_train, y_train)
print(mean_absolute_error(m.predict(X_test), y_test))
print(mean_squared_error(m.predict(X_test), y_test))
print(mean_squared_log_error(m.predict(X_test), y_test))
print(median_absolute_error(m.predict(X_test), y_test))
print('\n')

9.305891533247252
103.75391129947108
2.3355729712129665
8.61




# Какое качество получилось? Можно ли говорить о какой-либо значимой зависимости температуры воды от её химических показателей?

Качество после стандартизации данных стало хуже.
О зависимости температуры и химических показателях воды можно говорить.